<a href="https://colab.research.google.com/github/Mrunalika9/Real-Time-Crop-Stress/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit -q
import requests
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.utils.multiclass import unique_labels

# 🌾 Page Setup
st.set_page_config(page_title="Crop Stress Detector", layout="centered")
st.title("🌾 Real-Time Crop Stress Detection App")
st.write("Predict crop stress using real-time weather data and simulated NDVI values.")

# 📍 Get location input
place = st.text_input("📍 Enter a location name (e.g., Punjab, India):")

# Function to fetch coordinates
def get_coordinates(location_name):
    try:
        geolocator = Nominatim(user_agent="chaitu_crop_stress_detector")
        location = geolocator.geocode(location_name, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        st.error(f"Geocoding error: {e}")
        return None, None

if place:
    LAT, LON = get_coordinates(place)

    if LAT and LON:
        st.success(f"✅ Found coordinates: Latitude = {LAT}, Longitude = {LON}")

        # 🗓 Set date range (last 15 days)
        end_date = datetime.today()
        start_date = end_date - timedelta(days=15)
        START_DATE = start_date.strftime('%Y%m%d')
        END_DATE = end_date.strftime('%Y%m%d')

        st.write(f"📅 Fetching weather data from **{START_DATE}** to **{END_DATE}**")

        # 🌤 Fetch weather data
        NASA_API = (
            f"https://power.larc.nasa.gov/api/temporal/daily/point?"
            f"parameters=T2M,RH2M&community=AG&longitude={LON}"
            f"&latitude={LAT}&start={START_DATE}&end={END_DATE}&format=JSON"
        )

        response = requests.get(NASA_API)
        data = response.json()

        if 'properties' not in data or 'parameter' not in data['properties']:
            st.error("❌ Invalid API response. Check the location or date range.")
        else:
            daily_data = data['properties']['parameter']
            dates = list(daily_data['T2M'].keys())
            ndvi_values = np.random.uniform(0.2, 0.9, size=len(dates))  # Simulated NDVI

            records = []
            for i, date in enumerate(dates):
                temp = daily_data['T2M'][date]
                humidity = daily_data['RH2M'][date]
                ndvi = ndvi_values[i]
                rain = np.random.uniform(0, 15)

                if ndvi < 0.3 and temp > 35:
                    stress = 'High'
                elif ndvi > 0.6:
                    stress = 'Low'
                else:
                    stress = 'Moderate'

                records.append({
                    "date": date,
                    "temperature": temp,
                    "rainfall": rain,
                    "humidity": humidity,
                    "ndvi": ndvi,
                    "stress_level": stress
                })

            df = pd.DataFrame(records)

            st.subheader("🧾 Sample Weather + NDVI Data")
            st.dataframe(df)

            # 🧠 ML Processing
            X = df[["temperature", "rainfall", "humidity", "ndvi"]]
            y = df["stress_level"]

            label_encoder = LabelEncoder()
            y_encoded = label_encoder.fit_transform(y)

            # Safe split even with few samples
            try:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
                )
            except ValueError:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y_encoded, test_size=0.2, random_state=42
                )
                st.warning("⚠️ Not enough samples in some classes for stratify — using random split.")

            model = RandomForestClassifier(random_state=42)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

            labels_present = unique_labels(y_test, predictions)
            target_names_present = label_encoder.inverse_transform(labels_present)

            # 🧾 Show classification report
            report = classification_report(
                y_test, predictions,
                labels=labels_present,
                target_names=target_names_present,
                zero_division=0
            )
            st.subheader("📋 Classification Report")
            st.text(report)

            # 🔮 Predict today's stress level
            latest_input = df.iloc[[-1]][["temperature", "rainfall", "humidity", "ndvi"]]
            predicted_encoded = model.predict(latest_input)
            predicted_stress = label_encoder.inverse_transform(predicted_encoded)

            st.subheader(f"🌿 Predicted Stress Level for {df.iloc[-1]['date']}")
            st.success(f"**{predicted_stress[0]}**")

            # 📈 NDVI chart
            st.subheader("📊 NDVI Over Time")
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.bar(df['date'], df['ndvi'], color='green')
            ax.set_title('NDVI Over Time')
            ax.set_ylabel('NDVI')
            ax.set_xlabel('Date')
            plt.xticks(rotation=45)
            st.pyplot(fig)

    else:
        st.error("❌ Could not find the location.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


2025-04-08 13:36:29.238 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 13:36:29.239 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 13:36:29.379 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-08 13:36:29.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 13:36:29.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 13:36:29.383 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-08 13:36:29.384 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn